In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import pandas as pd
import numpy as np
import random
import os
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

In [40]:
df = pd.read_csv('/content/drive/MyDrive/irfinalproject/file.csv')

In [41]:
df

,Unnamed: 0,Label,Text,ProductId,UserId,Username,HelpfulnessNumerator,HelpfulnessDenominator,Score
0,300433,0,Yawn ... more anti-South cliches: Although the...,B000J2DHJA,A14HIHZX4LX3HA,"J. Wiley ""aka Angry CPU""",1.0,2.0,1.0
1,559025,0,Rated vs. Unrated: Don't make the mistake I ma...,B000REI2X6,A27VSP7HYF10DZ,Linda G. Smith,1.0,1.0,2.0
2,333148,0,"Maybe its just me, but: I didn't even get past...",B002CO180S,ABI9COIFMI7R8,"Kimberly A. Kempf ""momofone""",0.0,0.0,2.0
3,79440,0,Ouch! I've read term papers more interesting t...,B003TNANSO,A32FL8K77UAEN8,Ben Stupar,0.0,0.0,3.0
4,28605,0,Draco would not approve of this.: I am not som...,B00269YMNE,A3RWCDM8NY03SD,"Carol Robinson ""read too much""",2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
49995,46521,1,This product tastes just like regular peanut b...,B002GJ9JWS,A2U5A10E93A1S0,AJürckse,1.0,2.0,5.0
49996,450656,1,I have completely changed my diet to healthy e...,B0046HGTJ4,A1DG26ODRXIXBC,madame_dragon,0.0,0.0,5.0
49997,561485,0,I really like other coffee's and hot chocolate...,B000O393QK,A3MAFSTILQSOXR,"kiwi500 ""kiwi500""",0.0,0.0,3.0
49998,427752,1,I could spend a lot of time writing a very glo...,B0031VDJZS,A352U99Q6I1FE7,M.T.,0.0,0.0,5.0


# Users Recommendation

In [42]:
top_voted_users={}
least_voted_users={}
for i in range(len(df)):
  if df['HelpfulnessNumerator'].iloc[i] / df['HelpfulnessDenominator'].iloc[i] >= 0.8 :
    if(df['UserId'].iloc[i] in top_voted_users.keys()): 
      top_voted_users[df['UserId'].iloc[i]] += 1 
    else: 
      top_voted_users[df['UserId'].iloc[i]] = 1 
  elif df['HelpfulnessNumerator'].iloc[i] / df['HelpfulnessDenominator'].iloc[i] <=0.4 :
    if(df['UserId'].iloc[i] in least_voted_users.keys()): 
      least_voted_users[df['UserId'].iloc[i]] += 1 
    else: 
      least_voted_users[df['UserId'].iloc[i]] = 1 



<ipython-input-42-babb56d3b753>:4: RuntimeWarning: invalid value encountered in double_scalars
  if df['HelpfulnessNumerator'].iloc[i] / df['HelpfulnessDenominator'].iloc[i] >= 0.8 :
<ipython-input-42-babb56d3b753>:9: RuntimeWarning: invalid value encountered in double_scalars
  elif df['HelpfulnessNumerator'].iloc[i] / df['HelpfulnessDenominator'].iloc[i] <=0.4 :


In [43]:
## Sorting top recommended users dictionary
keys = list(top_voted_users.keys())
values = list(top_voted_users.values())
sorted_value_index = np.argsort(values)[::-1]
top_voted_users = {keys[i]: values[i] for i in sorted_value_index}

In [44]:
## Sorting least recommended users dictionary

keys = list(least_voted_users.keys())
values = list(least_voted_users.values())
sorted_value_index = np.argsort(values)[::-1]
least_voted_users = {keys[i]: values[i] for i in sorted_value_index}

In [45]:
### print top 10 most recommended users whose helpfulness score >3
import itertools
top_users = dict(itertools.islice(top_voted_users.items(), 10))
print("    User        :  No of recommendations")
top_users

    User        :  No of recommendations


{'A1YUL9PCJR3JTY': 25,
 'A3HPCRD9RX351S': 21,
 'A3OXHLG6DIBRW8': 17,
 'AY12DBB0U420B': 15,
 'A29JUMRL1US6YP': 14,
 'AQQLWCMRNDFGI': 13,
 'A281NPSIMI1C2R': 12,
 'A2MUGFV2TDQ47K': 11,
 'AEC90GPFKLAAW': 11,
 'A1LZJZIHUPLDV4': 9}

In [46]:
### print top 10 least recommended users whose helpfulness score <3
least_rec_users = dict(itertools.islice(least_voted_users.items(), 10))
print("    User        :  No of recommendations")
least_rec_users


    User        :  No of recommendations


{'A1AEQZM99LO9VA': 7,
 'A2FRFAQCWZJT3Q': 6,
 'A2MUGFV2TDQ47K': 6,
 'A1Z7Y2GMAP9SRY': 5,
 'A1TMAVN4CEM8U8': 4,
 'A1KJ1T6UQW2TFE': 4,
 'A2NI99G9BANORP': 4,
 'A3EPJSPH03AS29': 4,
 'A27HB4L3I1WJUR': 4,
 'A1SCANWWQTEG9I': 4}

# Product Recommendation

In [47]:
print('Number of unique USERS in Raw data = ', df['UserId'].nunique())
print('Number of unique ITEMS in Raw data = ', df['ProductId'].nunique())

Number of unique USERS in Raw data =  39494
Number of unique ITEMS in Raw data =  20604


In [48]:
most_rated = df.groupby('UserId').size().sort_values(ascending=False)[:10]
most_rated

UserId
A3OXHLG6DIBRW8    46
AY12DBB0U420B     40
A1YUL9PCJR3JTY    39
A281NPSIMI1C2R    33
A2MUGFV2TDQ47K    23
A3TVZM3ZIXG8YW    22
A3HPCRD9RX351S    22
A1Z54EM24Y40LL    22
A2FRFAQCWZJT3Q    21
A1UQBFCERIP7VJ    21
dtype: int64

In [49]:
counts = df['UserId'].value_counts()
df_final = df[df['UserId'].isin(counts[counts >= 5].index)]

In [50]:
print('Number of users who have rated 5 or more items =', len(df_final))
print('Number of unique USERS in final data = ', df_final['UserId'].nunique())
print('Number of unique ITEMS in final data = ', df_final['ProductId'].nunique())

Number of users who have rated 5 or more items = 3064
Number of unique USERS in final data =  421
Number of unique ITEMS in final data =  2181


In [51]:
final_ratings_matrix = pd.pivot_table(df_final,index=['UserId'], columns = 'ProductId', values = "Score")
final_ratings_matrix.fillna(0,inplace=True)
print('Shape of final_ratings_matrix: ', final_ratings_matrix.shape)
given_num_of_ratings = np.count_nonzero(final_ratings_matrix)
print('given_num_of_ratings = ', given_num_of_ratings)
possible_num_of_ratings = final_ratings_matrix.shape[0] * final_ratings_matrix.shape[1]
print('possible_num_of_ratings = ', possible_num_of_ratings)
density = (given_num_of_ratings/possible_num_of_ratings)
density *= 100
print ('density: {:4.2f}%'.format(density))

Shape of final_ratings_matrix:  (421, 2181)
given_num_of_ratings =  2935
possible_num_of_ratings =  918201
density: 0.32%


In [52]:
final_ratings_matrix.tail()

ProductId,7310172101,B00004RYGX,B000084DWM,B000084EHL,B000084EJT,B000084EK5,B000084EKC,B000084ETV,B000084F1I,B00008CQVA,...,B0093A5QMA,B00954NY46,B00975HC9G,B009858H6M,B009AFJ548,B009B87SAC,B009EXL6IO,B009GHI5Q4,B009K2BBQG,B009M2LUEW
UserId,,,,,,,,,,,,,,,,,,,,,
AYGJ96W5KQMUJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
AYOMAHLWRQHUG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AYWUHB7N8XGZQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZU4M5K1N2LAB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZV26LP92E6WU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
final_ratings_matrix_T = final_ratings_matrix.transpose()
final_ratings_matrix_T.head()

UserId,A10G136JEISLVR,A10PEXB6XAQ5XF,A10PJEHY3JKKQG,A10ZBR6O8S8OCY,A11ED8O95W2103,A11OTLEDSW8ZXD,A11UPNFYDICF3C,A11YOTONCPRQ9S,A12ENBT314RFXR,A132ETQPMHQ585,...,AXJGCAD36N915,AXO4PQU0XG3TG,AY12DBB0U420B,AY1EF0GOH80EK,AYB4ELCS5AM8P,AYGJ96W5KQMUJ,AYOMAHLWRQHUG,AYWUHB7N8XGZQ,AZU4M5K1N2LAB,AZV26LP92E6WU
ProductId,,,,,,,,,,,,,,,,,,,,,
7310172101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004RYGX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000084DWM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000084EHL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000084EJT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
train_data, test_data = train_test_split(df_final, test_size = 0.3, random_state=0)

In [55]:
train_data_grouped = train_data.groupby('ProductId').agg({'UserId': 'count'}).reset_index()
train_data_grouped.rename(columns = {'UserId': 'score'},inplace=True)
train_data_grouped.head()

,ProductId,score
0,7310172101,2
1,B00004RYGX,2
2,B000084EHL,1
3,B000084EJT,1
4,B000084EKC,1


In [56]:
#Sort the products on recommendation score 
train_data_sort = train_data_grouped.sort_values(['score', 'ProductId'], ascending = [0,1]) 
      
#Generate a recommendation rank based upon score 
train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
#Get the top 5 recommendations 
popularity_recommendations = train_data_sort.head(5) 
popularity_recommendations 

,ProductId,score,Rank
1475,B005VOOLXM,8,1.0
655,B001BCXTGS,7,2.0
1012,B002IEZJMA,7,3.0
1437,B005HG9ERW,7,4.0
1542,B007I7YZJK,7,5.0


In [57]:
# Use popularity based recommender model to make predictions
def recommend(user_id):     
    user_recommendations = popularity_recommendations 
          
    #Add user_id column for which the recommendations are being generated 
    user_recommendations['UserId'] = user_id 
      
    #Bring user_id column to the front 
    cols = user_recommendations.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    user_recommendations = user_recommendations[cols] 
          
    return user_recommendations

In [58]:
find_recom = [15,121,200]   # This list is user choice.
for i in find_recom:
    print("Here is the recommendation for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n") 

Here is the recommendation for the userId: 15

      UserId   ProductId  score  Rank
1475      15  B005VOOLXM      8   1.0
655       15  B001BCXTGS      7   2.0
1012      15  B002IEZJMA      7   3.0
1437      15  B005HG9ERW      7   4.0
1542      15  B007I7YZJK      7   5.0


Here is the recommendation for the userId: 121

      UserId   ProductId  score  Rank
1475     121  B005VOOLXM      8   1.0
655      121  B001BCXTGS      7   2.0
1012     121  B002IEZJMA      7   3.0
1437     121  B005HG9ERW      7   4.0
1542     121  B007I7YZJK      7   5.0


Here is the recommendation for the userId: 200

      UserId   ProductId  score  Rank
1475     200  B005VOOLXM      8   1.0
655      200  B001BCXTGS      7   2.0
1012     200  B002IEZJMA      7   3.0
1437     200  B005HG9ERW      7   4.0
1542     200  B007I7YZJK      7   5.0




<ipython-input-57-0ececd569596>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations['UserId'] = user_id
<ipython-input-57-0ececd569596>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations['UserId'] = user_id
<ipython-input-57-0ececd569596>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [59]:
df_CF = pd.concat([train_data, test_data]).reset_index()
df_CF.tail()

,index,Unnamed: 0,Label,Text,ProductId,UserId,Username,HelpfulnessNumerator,HelpfulnessDenominator,Score
3059,12403,431504,0,It's Okay: Well even thought I think this seco...,B003ZMFUKU,A2KZRK2T9KFA0X,Sarah,0.0,0.0,2.0
3060,47218,2857,1,Not only does Vita coco have the best taste .....,B0085G4ACA,A2YHXAZLCLDT8D,"Mark Smith ""Food lover""",0.0,0.0,5.0
3061,30584,41924,0,"boring for an 8th grader, good for a history l...",B006Q7YFLQ,A19XBW5WUDH1Z,David Nox,0.0,0.0,3.0
3062,49580,326831,1,Great stuff. Didn't expect them to taste as go...,B0014JUPTW,A198JRUIP09HX5,"V. Gera ""deal hound""",0.0,0.0,5.0
3063,39982,228152,1,"To me, this has a good, balanced flavor. Some...",B003VXHGPK,A35OY2ASP1MP5M,Lyn Barr,0.0,0.0,5.0


In [60]:
#User-based Collaborative Filtering
# Matrix with row per 'user' and column per 'item' 
pivot_df = pd.pivot_table(df_CF,index=['UserId'], columns = 'ProductId', values = "Score")
pivot_df.fillna(0,inplace=True)
print(pivot_df.shape)
pivot_df.head()

(421, 2181)


ProductId,7310172101,B00004RYGX,B000084DWM,B000084EHL,B000084EJT,B000084EK5,B000084EKC,B000084ETV,B000084F1I,B00008CQVA,...,B0093A5QMA,B00954NY46,B00975HC9G,B009858H6M,B009AFJ548,B009B87SAC,B009EXL6IO,B009GHI5Q4,B009K2BBQG,B009M2LUEW
UserId,,,,,,,,,,,,,,,,,,,,,
A10G136JEISLVR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10PEXB6XAQ5XF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10PJEHY3JKKQG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10ZBR6O8S8OCY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11ED8O95W2103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
pivot_df['user_index'] = np.arange(0, pivot_df.shape[0], 1)
pivot_df.head()

ProductId,7310172101,B00004RYGX,B000084DWM,B000084EHL,B000084EJT,B000084EK5,B000084EKC,B000084ETV,B000084F1I,B00008CQVA,...,B00954NY46,B00975HC9G,B009858H6M,B009AFJ548,B009B87SAC,B009EXL6IO,B009GHI5Q4,B009K2BBQG,B009M2LUEW,user_index
UserId,,,,,,,,,,,,,,,,,,,,,
A10G136JEISLVR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
A10PEXB6XAQ5XF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
A10PJEHY3JKKQG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
A10ZBR6O8S8OCY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
A11ED8O95W2103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [62]:
pivot_df.set_index(['user_index'], inplace=True)

# Actual ratings given by users
pivot_df.head()

ProductId,7310172101,B00004RYGX,B000084DWM,B000084EHL,B000084EJT,B000084EK5,B000084EKC,B000084ETV,B000084F1I,B00008CQVA,...,B0093A5QMA,B00954NY46,B00975HC9G,B009858H6M,B009AFJ548,B009B87SAC,B009EXL6IO,B009GHI5Q4,B009K2BBQG,B009M2LUEW
user_index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# Convert dense matrix to CSR sparse matrix
pivot_csr = csr_matrix(pivot_df)

# Singular Value Decomposition
U, sigma, Vt = svds(pivot_csr, k=50)

# Construct diagonal array in SVD
sigma = np.diag(np.float32(sigma))

In [64]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 

# Predicted ratings
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot_df.columns)
preds_df.head()

ProductId,7310172101,B00004RYGX,B000084DWM,B000084EHL,B000084EJT,B000084EK5,B000084EKC,B000084ETV,B000084F1I,B00008CQVA,...,B0093A5QMA,B00954NY46,B00975HC9G,B009858H6M,B009AFJ548,B009B87SAC,B009EXL6IO,B009GHI5Q4,B009K2BBQG,B009M2LUEW
0,1.537731e-15,-4.462308e-18,-9.248547e-18,-1.662763e-18,2.080849e-16,1.558854e-16,2.075589e-16,2.613743e-17,-7.456728e-19,4.326242e-17,...,1.721644e-18,-2.079743e-16,-1.804441e-17,-8.396276e-18,-1.079763e-15,4.701216e-17,2.594881e-16,4.128703e-17,3.168175e-17,8.456463e-16
1,8.733504e-16,1.201107e-03,5.541771e-03,2.298902e-17,-3.739098e-03,-2.804324e-03,-3.739098e-03,3.495191e-03,-2.316586e-05,-3.024559e-03,...,8.084707e-19,-8.768531e-03,-1.228067e-17,5.645744e-19,4.425968e-03,-9.993005e-03,-4.673873e-03,-3.568425e-03,-2.419316e-02,-4.645219e-02
2,9.820670e-15,-1.071442e-16,-2.238865e-16,-9.285343e-19,-2.092590e-16,-1.570438e-16,-2.095527e-16,1.455156e-16,-3.563607e-18,9.309558e-17,...,9.614154e-19,-7.365320e-16,-1.007651e-17,-2.250240e-16,-3.566463e-15,1.013000e-15,-2.619189e-16,3.610847e-16,-3.275363e-16,4.100090e-15
3,3.330900e-15,1.379026e-03,1.373860e-02,-1.502138e-17,2.191182e-03,1.643387e-03,2.191182e-03,1.562905e-03,3.325303e-05,-1.228956e-02,...,-4.091884e-18,1.301818e-02,-4.469152e-17,5.026547e-16,7.436957e-03,-2.682560e-02,2.738978e-03,-9.531562e-03,-3.232425e-02,-4.225264e-02
4,8.881784e-16,1.194001e-17,-1.567676e-17,2.596647e-31,3.611528e-17,2.708646e-17,3.611528e-17,1.922784e-18,-4.148650e-19,5.970556e-16,...,-5.324590e-32,2.356984e-16,-4.080199e-31,3.261691e-16,4.537013e-16,-1.107640e-16,4.514409e-17,-3.587527e-17,-7.164045e-17,-5.094686e-16


In [65]:
def recommend_items(userID, pivot_df, preds_df, num_recommendations):
      
    user_idx = userID-1 # index starts at 0
    
    # Get and sort the user's ratings
    sorted_user_ratings = pivot_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_ratings
    sorted_user_predictions = preds_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_predictions

    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis=1)
    temp.index.name = 'Recommended Items'
    temp.columns = ['user_ratings', 'user_predictions']
    
    temp = temp.loc[temp.user_ratings == 0]   
    temp = temp.sort_values('user_predictions', ascending=False)
    print('\nBelow are the recommended items for user(user_id = {}):\n'.format(userID))
    print(temp.head(num_recommendations))

In [66]:
#Enter 'userID' and 'num_recommendations' for the user #
userID = 121
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 121):

                   user_ratings  user_predictions
Recommended Items                                
B005HG9ERW                  0.0      9.874613e-16
B003TNANSO                  0.0      9.018889e-16
B001AHFVHO                  0.0      8.709544e-16
B001BDDTB2                  0.0      7.971822e-16
B003NUYBV8                  0.0      7.572608e-16


In [67]:
rmse_df = pd.concat([final_ratings_matrix.mean(), preds_df.mean()], axis=1)
rmse_df.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']
print(rmse_df.shape)
rmse_df['item_index'] = np.arange(0, rmse_df.shape[0], 1)
rmse_df.head()

(2181, 2)


,Avg_actual_ratings,Avg_predicted_ratings,item_index
ProductId,,,
7310172101,0.009501,9.501188e-03,0
B00004RYGX,0.014252,8.102515e-03,1
B000084DWM,0.011876,8.440533e-03,2
B000084EHL,0.011876,-2.843328e-17,3
B000084EJT,0.009501,1.213695e-02,4


In [68]:
RMSE = round((((rmse_df.Avg_actual_ratings - rmse_df.Avg_predicted_ratings) ** 2).mean() ** 0.5), 5)
print('\nRMSE SVD Model = {} \n'.format(RMSE))


RMSE SVD Model = 0.00756 



In [69]:
 # Enter 'userID' and 'num_recommendations' for the user #
userID = 200
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 200):

                   user_ratings  user_predictions
Recommended Items                                
B000E63LDS                  0.0          0.586030
B005CGUUJS                  0.0          0.452051
B004W19VBM                  0.0          0.452051
B000W5N12C                  0.0          0.452051
B003H2RQ82                  0.0          0.361641


# Product Recommendation using Rouge Score

In [102]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [109]:
pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [112]:
from rouge import Rouge
from textblob import TextBlob

r = Rouge()

In [126]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [132]:
query= input()
query = str(TextBlob(query).correct())      ### spelling correction

i love cofeeee.


In [133]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(query)

filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

filtered_sentence = []

for w in word_tokens:
	if w not in stop_words:
		filtered_sentence.append(w)


print(word_tokens)
print(filtered_sentence)

q = ""
for w in filtered_sentence:
  q = q + w +" "

['i', 'love', 'coffee', '.']
['love', 'coffee', '.']


In [134]:
list=[]
for i in range(len(df['Text'])):
  d = r.get_scores(df['Text'].iloc[i],query)[0]
  if(d['rouge-1']['f'] > 0.15):
    list.append([d['rouge-l']['f'],i])
    
new_list=sorted(list,key=lambda d: d[1])

In [137]:
s = {df['ProductId'].iloc[new_list[0][1]]} ### set to store unique product ids
i = 1
while (len(s)<5):
  # print(df['Text'].iloc[new_list[i][1]])
  s.add(df['ProductId'].iloc[new_list[i][1]])
  i+=1

print(s)    

{'B001FKC4XA', 'B000E682LY', 'B0000TA2TK', 'B001EQ4Q8Q', 'B000HDK0D2'}
